This is a slightly involved classification notebook - using superpixels to generate features. The results look good in that we reduce commission errors. The segmentation might be 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tqdm import tqdm
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from skimage.segmentation import felzenszwalb
from skimage.color import label2rgb
from rscube.nd_tools import (get_array_from_features, 
                             get_features_from_array, 
                             get_superpixel_area_as_features, 
                             get_superpixel_means_as_features,
                             get_superpixel_stds_as_features, 
                             scale_img)
from rscube.rio_tools import (get_geopandas_features_from_array, 
                              rasterize_shapes_to_array, 
                              get_indices_from_extent, get_cropped_profile)

from sklearn.cluster import KMeans
from skimage.segmentation import felzenszwalb
from skimage.color import label2rgb
from pprint import pprint
import geopandas as gpd
from rasterio import plot
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import geopandas as gpd
from shapely.geometry import box
from rasterio.windows import Window
from pathlib import Path
from rasterio.crs import CRS
from dem_stitcher.rio_tools import reproject_arr_to_match_profile
import joblib

Currently, we don't have a way to link chips and planet IDs we are using so this is manual.

In [ ]:
PLANET_ID = '20211003_161639_91_241d'

In [ ]:
planet_image_path = Path(f'local_chips/{PLANET_ID}_3B_AnalyticMS_SR_8b.tif')
planet_image_path.exists()

# Path

In [ ]:
out_class_dir = Path(f'classification_outputs/{PLANET_ID}_classification')
out_class_dir.mkdir(exist_ok=True, parents=True)

# Load Chips

Want to crop image based on chip

In [ ]:
df = gpd.read_file('chips.geojson')
df.head()

In [ ]:
with rasterio.open(planet_image_path) as ds:
    image_crs = ds.crs
    image_box = box(*ds.bounds)
    image_bounds = list(ds.bounds)
    image_profile = ds.profile
    image_shape = image_profile['height'], image_profile['width']

image_shape

In [ ]:
df_utm = df.to_crs(image_crs)
intersects = df_utm.geometry.intersects(image_box)
df_chip = df_utm[intersects].reset_index(drop=True)
df_chip.head()

It's a little weird because all Planet Imagery is in UTM and Chips are in Lon/Lat.

In [ ]:
(start_y, start_x), (stop_y, stop_x) = get_indices_from_extent(image_profile['transform'],
                                                               list(df_chip.total_bounds),
                                                               shape=image_shape)
window = Window.from_slices((start_y, stop_y), (start_x, stop_x))
sx, sy = np.s_[start_x: stop_x], np.s_[start_y: stop_y]
profile_cropped = get_cropped_profile(image_profile, sx, sy)

We are going to label things based on the chip index.

In [ ]:
index = df_chip.random_id[0]
index

In [ ]:
with rasterio.open(planet_image_path) as ds:
    image_c = ds.read(window=window)
    image_c = image_c.transpose([1, 2, 0]).astype(np.float32)
image_c.shape

In [ ]:
with rasterio.open(planet_image_path) as ds:

    t = ds.tags()
    d = ds.descriptions
    
list(enumerate(d))

In [ ]:
mask = (image_c[..., 0] == image_profile['nodata'])
image_c[mask, :] = np.nan

In [ ]:
rgb = scale_img(image_c[..., [7, 5, 3]])
# plt.imshow(rgb)

In [ ]:
image = image_c[..., [7, 5, 3]]

image_view = image.copy()
for k in tqdm(range(3)):
    m0 = np.nanpercentile(image[~mask, k], 2)
    m1 = np.nanpercentile(image[~mask, k], 98)
    image_view[~mask, k] = np.clip(image[~mask, k], m0, m1)

In [ ]:
rgb = scale_img(image_view)
plt.imshow(rgb)

In [ ]:
p_cropped = profile_cropped.copy()
p_cropped['count'] = 3
p_cropped['dtype'] = 'float32'
p_cropped['nodata'] = np.nan
with rasterio.open(out_class_dir / f'cropped_to_chip_{index}.tif', 'w', **p_cropped) as ds:
    ds.write(rgb.transpose([2, 0, 1]))

You could use the above to figure out some training data. We label it `training_data_{index}`, according to the chip ID.

# Segmentation

Going to use multiscale superpixels - see [this paper](https://www.mdpi.com/2072-4292/12/12/2048) and the references. I learned about them [here](https://link.springer.com/chapter/10.1007/978-94-017-7239-6_8).

Best discussion of felzenswalb algorithm (and parameters below) is by a fellow JPL team: https://ieeexplore.ieee.org/document/5593215

In [ ]:
def get_segmentation(image, min_size):
    mask = np.isnan(image[..., 0])
    image_seg = image.copy()
    # So that np.nans don't give us problems
    image_seg[mask, :] = -10_000
    segments_fz = felzenszwalb(image_seg,
                               # may want to play with this
                               scale=.1, 
                               # normally gaussian filter is applied - can experiment
                               sigma=0.2,
                               # minimum size of segments
                               min_size=min_size
                              )
    return segments_fz

We are going to use the RGB image from above.

In [ ]:
get_segmentation_partial = lambda min_size: get_segmentation(rgb, min_size)
segmentations = list(map(get_segmentation_partial, tqdm([5, 20, 50])))

In [ ]:
X = segmentations[0].copy()
X[mask] = 0
#superpixel_labels_viz = label2rgb(X, bg_label=0)
#plt.imshow(superpixel_labels_viz)

Below would be used to save the segments to a vector file. It takes some time and the segments can be approximately ~1 GB.

In [ ]:
segments_fz = segmentations[1]

In [ ]:
%%time

# features = get_geopandas_features_from_array(segments_fz.astype(np.int32), 
#                                              profile['transform'], 
#                                              label_name='label')

In [ ]:
%%time

# df_segments = gpd.GeoDataFrame.from_features(features)
# df_segments.head()

In [ ]:
%%time

# df_segments.crs = profile['crs']
# df_segments.to_file('segments.geojson', driver='GeoJSON')

# Generate Features

In [ ]:
rgb.shape

In [ ]:
pixel_features = rgb.reshape((-1, rgb.shape[-1]))

In [ ]:
multi_superpixel_features_mean = [get_superpixel_means_as_features(seg, rgb) for seg in tqdm(segmentations)]

In [ ]:
multi_superpixel_features_mean[0].shape

In [ ]:
multi_superpixel_features_std = [get_superpixel_stds_as_features(seg, rgb) for seg in tqdm(segmentations)]

In [ ]:
multi_superpixel_means = [get_array_from_features(seg, feature) 
                          for (seg, feature) in zip(tqdm(segmentations),                                                                                                     multi_superpixel_features_mean)]
multi_superpixel_stds = [get_array_from_features(seg, feature) 
                          for (seg, feature) in zip(tqdm(segmentations),                                                                                                     multi_superpixel_features_std)]

In [ ]:
scale_ind = 0
img_super = multi_superpixel_means[scale_ind]
p_superp = p_cropped.copy()
p_superp['count'] = 3

with rasterio.open(out_class_dir / f'superpixel_means_{scale_ind}_{index}.tif', 'w', **p_superp) as ds:
    ds.write(img_super.transpose([2, 0, 1]))

In [ ]:
# scale_ind = 0
# img_super = multi_superpixel_stds[scale_ind]

# with rasterio.open(out_class_dir / f'superpixel_stds_{scale_ind}_{index}.tif', 'w', **p_superp) as ds:
#     ds.write(img_super.transpose([2, 0, 1]))

In [ ]:
superpixel_means_pixel_f = [superpixel_means.reshape((-1, rgb.shape[-1])) 
                            for superpixel_means in multi_superpixel_means]
superpixel_stds_pixel_f = [superpixel_stds.reshape((-1, rgb.shape[-1])) 
                            for superpixel_stds in multi_superpixel_stds]

In [ ]:
all_features = np.hstack(([pixel_features]  
                          + superpixel_means_pixel_f 
                          + superpixel_stds_pixel_f
                         ))
all_features.shape

# Generate Training Data

Using Peckel Occurance data to generate training data

In [ ]:
from osgeo import gdal
from dem_stitcher.rio_window import read_raster_from_window
from rasterio.warp import transform_bounds

def build_peckel_vrt(extent: list, 
                     out_path: Path):
    df_peckel_data = gpd.read_file('peckel_tiles.geojson')
    bbox = box(*extent)
    ind_inter = df_peckel_data.geometry.intersects(bbox)
    df_subset = df_peckel_data[ind_inter].reset_index(drop=True)
    gdal.BuildVRT(str(out_path), df_subset.source_url.tolist())
    return out_path

def get_peckel_raster(extent:list) -> tuple:
    tmp_vrt = Path('peckel_data_tmp.vrt')
    build_peckel_vrt(extent, tmp_vrt)
    X, p = read_raster_from_window(tmp_vrt,
                                   extent,
                                   CRS.from_epsg(4326))
    tmp_vrt.unlink()
    p['driver'] = 'GTiff'
    return X, p

In [ ]:
df_chip.to_crs(4326).total_bounds

In [ ]:
X_occ, p_occ = get_peckel_raster(df_chip.to_crs(4326).total_bounds)

In [ ]:
with rasterio.open(out_class_dir / f'occurence_c{index}.tif', 'w', **p_occ) as ds:
    ds.write(X_occ, 1)

In [ ]:
#plt.imshow(X_occ)

In [ ]:
from scipy import ndimage

OCC_MIN = 30
DIST_TO_EXTENT = 5

water_ind = (X_occ >= OCC_MIN)
water_extent = (X_occ > 0)

water_extent_arr = (~water_extent).astype(int)
water_dist = ndimage.distance_transform_edt(water_extent_arr)

land_ind = water_dist > DIST_TO_EXTENT

In [ ]:
# fig, ax = plt.subplots(1, 3, figsize=(15,5))

# ax[0].imshow(water_ind)
# ax[0].set_title('Water Mask')

# ax[1].imshow(water_dist)
# ax[1].set_title('Distance to non-zero occurence')

# ax[2].imshow(land_ind)
# ax[2].set_title('Land Mask')

In [ ]:
training_data_4326 = np.zeros(X_occ.shape) * np.nan
training_data_4326[land_ind] = 2
training_data_4326[water_ind] = 1
#plt.title('np.nan = no label/nodata, 1=water, 2 = land')
#plt.imshow(training_data_4326, interpolation='none')

In [ ]:
p_temp = p_occ.copy()
p_temp['dtype'] = 'float32'
p_temp['nodata'] = np.nan


training_data_r, p_r = reproject_arr_to_match_profile(training_data_4326, p_temp, p_cropped)

training_data_r = training_data_r[0, ...].round()

#plt.imshow(training_data_r, interpolation='none')

In [ ]:
with rasterio.open(out_class_dir/'training_data_reprojected.tif', 'w', **p_r) as ds:
    ds.write(training_data_r, 1)

In [ ]:
labels = training_data_r.copy()

In [ ]:
labels_f = labels.reshape((-1, 1)).ravel()

f_mask = (~((labels_f == 0) | np.isnan(labels_f) | mask.ravel())).ravel()

In [ ]:
rf_path = Path('rf.joblib')
if rf_path.exists():
    rf_prev = joblib.load(rf_path)
    
    
rf = RandomForestClassifier(n_estimators=500,
                            oob_score=True,
                            random_state=0,
                            n_jobs=8,
                            warm_start=True)
rf.planet_ids = []


There must be a better way to do this - this makes sure training data has equal water and land, the later which is much more over-represented. Think something like this will do: https://imbalanced-learn.org/stable/auto_examples/applications/plot_multi_class_under_sampling.html#sphx-glr-auto-examples-applications-plot-multi-class-under-sampling-py/

In [ ]:
X_labeled_all = all_features[f_mask]
y_labeled_all = labels_f[f_mask]

In [ ]:


if (1 in np.unique(y_labeled_all)):
    PERCENT_OF_DATA = .05
    subset_of_data_pixels = (~f_mask).sum() * PERCENT_OF_DATA

    np.random.seed(0)
    water_ind_f = np.where(y_labeled_all == 1)[0]

    # Make sure training water never exceed more than p% of available data
    n_max = int(np.minimum(y_labeled_all.shape[0], subset_of_data_pixels))
    water_ind_f_sample = np.random.choice(water_ind_f, size=n_max)

    land_ind_f = np.where(y_labeled_all == 2)[0]
    n_land = int(np.minimum(water_ind_f_sample.shape[0], subset_of_data_pixels)) 
    land_ind_f_sample = np.random.choice(land_ind_f, size=n_land)

    indices_for_labeling = np.hstack([water_ind_f_sample, land_ind_f_sample])
    indices_for_labeling.shape

    y_labeled = y_labeled_all[indices_for_labeling].astype(int)
    X_labeled = X_labeled_all[indices_for_labeling, ...]

else:
    print(f'No water in training data: {1 not in np.unique(y_labeled_all)}')
    print(f'We previously trained on this Planet chip: {PLANET_ID not in rf.planet_ids}')
    
    PERCENT_OF_DATA = .1
    land_ind_f = np.where(y_labeled_all == 2)[0]
    n = int(PERCENT_OF_DATA * (~f_mask).sum())
    land_ind_f_sample = np.random.choice(land_ind_f, size=n)
    
    
    y_labeled = y_labeled_all[land_ind_f_sample].astype(int)
    X_labeled = X_labeled_all[land_ind_f_sample, ...]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_labeled, 
                                                    y_labeled, 
                                                    test_size=0.2,
                                                    train_size=0.8,
                                                    random_state=0,
                                                    # stratify=y_labeled
                                                    )

In [ ]:
%%time

print(f'The Peckel data has water: {1 in np.unique(y_labeled_all)}')
print(f'We previously trained on this Planet chip: {PLANET_ID not in rf.planet_ids}')

if (1 not in np.unique(y_labeled_all)):
    rf = rf_prev
    # see this discussion: https://stackoverflow.com/questions/42757892/how-to-use-warm-start
    # print(f'loading previously trained random forrest with {rf.n_estimators} trees and adding 250 more')

    # rf.set_params(n_estimators=(rf.n_estimators + 250))
    # print(f'now {rf.n_estimators} trees')
else:  
    rf.fit(X_train, 
           y_train)


In [ ]:
%%time

y_pred = rf.predict(X_test)

In [ ]:
class_dict = {1: 'water', 
              2: 'not_water'}

In [ ]:
y_pred_str = pd.Series([class_dict[class_id] for class_id in y_pred], name='Predicted')
y_true_str = pd.Series([class_dict[class_id] for class_id in y_test], name='True')

This gives us how well we did on the labeled test data. Random forests will do pretty well on the labeled data since the labeled data is continugous and the superpixels aggregations will likely be shared across pixels.

In [ ]:
df_conf = pd.crosstab(y_true_str, y_pred_str)
df_conf

In [ ]:
all_mask_f = (mask.reshape((-1,)))

X_all = all_features[~all_mask_f, 
                     ...]

In [ ]:
%%time

y_all = rf.predict(X_all)

In [ ]:
y_arr = np.zeros(mask.shape)

y_arr[~mask] = y_all
y_arr[mask] = 0

In [ ]:
p = profile_cropped.copy()
p['count'] = 1 
p['dtype'] = np.uint8
with rasterio.open(out_class_dir / f'classification_chip-{index}_planet-{PLANET_ID}.tif', 'w', **p) as ds:
    ds.write(y_arr.astype(np.uint8), 1)

In [ ]:
plt.imshow(y_arr, interpolation='none')

# Feature Importances

See: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.feature_importances_

In [ ]:
rf.feature_importances_

In [ ]:
joblib.dump(rf, out_class_dir/f'rf_{PLANET_ID}.joblib') 

# Save Data

In [ ]:
chip_data = df_chip.to_dict('record')[0]
chip_data

In [ ]:
# not_water_class, water_class = df_conf.to_dict('records')
# not_water_class

In [ ]:
fi = list(map(lambda x: f'{x:06f}', rf.feature_importances_))

In [ ]:
class_data = {'geometry': chip_data['geometry'],
              'feature_list': 'pixels (rgb) ,mean_superpixel (rgb for 3 segs),std_superpixels for rgb image (rgb for 3 segs)',
              'feature_importances': ','.join(fi),
#               'accuracy': {'not_water_class_true': not_water_class,
#                            'water_class_true': water_class},
              's3_bucket': '',
              's3_key': '',
              'chip_id': chip_data['random_id'],
              'strata': chip_data['STRATA']
             }
class_data

In [ ]:
df_out = gpd.GeoDataFrame([class_data],
                         crs=df_chip.crs)

In [ ]:
df_out.to_file(out_class_dir / f'classification_chip-{index}_planet-{PLANET_ID}.geojson', driver='GeoJSON')


# Save Running Model

In [ ]:
rf.planet_ids.append(PLANET_ID)
joblib.dump(rf, f'rf.joblib') 

rf = None